In [1]:
import pandas as pd
import numpy as np
import seaborn as sns;sns.set(style="white")
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score


In [2]:
df = pd.read_csv("ucm_week_sun.csv")
# new column 'total_paid' added 
df.rename(columns = {'count(DISTINCT COLL_NUM)':'donations'}, inplace = True)
df.shape

(260912, 4)

In [3]:
# new column 'total_paid' added 
df['total_paid'] = df['GRIFOLS_TOTAL_PAID'] * df['donations']
df.head()

,site_code,week_end_sun,GRIFOLS_TOTAL_PAID,donations,total_paid
0,601,2022-12-11,80.0,11,880.0
1,380,2022-07-10,40.0,544,21760.0
2,914,2022-07-24,50.0,336,16800.0
3,71,2022-11-13,55.0,6,330.0
4,610,2022-11-06,100.0,212,21200.0


In [4]:
#df.isna().sum()
#df.dropna(inplace=True, axis=0)
df["GRIFOLS_TOTAL_PAID"] = pd.cut(df["GRIFOLS_TOTAL_PAID"], bins=[0, 30, 40, 50, 80, 100, 120, 150, float('inf')], 
                                  labels=['amt_0_30', 'amt_30_40','amt_40_50','amt_50_80','amt_80_100','amt_100_120','amt_120_150','amt_150+'])

df.head()

,site_code,week_end_sun,GRIFOLS_TOTAL_PAID,donations,total_paid
0,601,2022-12-11,amt_50_80,11,880.0
1,380,2022-07-10,amt_30_40,544,21760.0
2,914,2022-07-24,amt_40_50,336,16800.0
3,71,2022-11-13,amt_50_80,6,330.0
4,610,2022-11-06,amt_80_100,212,21200.0


In [5]:
df_avg_price = df.groupby(['site_code','week_end_sun','GRIFOLS_TOTAL_PAID']).agg({'donations': ['sum'],'total_paid':['sum']})
df_avg_price.columns = ['total_donations','total_paid_amount']
df_avg_price = df_avg_price.reset_index()
df_avg_price['avg_price'] = df_avg_price['total_paid_amount'] / (df_avg_price['total_donations'] + 0.001)
df_avg_price

,site_code,week_end_sun,GRIFOLS_TOTAL_PAID,total_donations,total_paid_amount,avg_price
0,5,2022-01-02,amt_0_30,303,9060.0,29.900891
1,5,2022-01-02,amt_30_40,12,440.0,36.663611
2,5,2022-01-02,amt_40_50,1,50.0,49.950050
3,5,2022-01-02,amt_50_80,6,480.0,79.986669
4,5,2022-01-02,amt_80_100,195,19500.0,99.999487
...,...,...,...,...,...,...
129739,916,2023-01-01,amt_50_80,0,0.0,0.000000
129740,916,2023-01-01,amt_80_100,286,28390.0,99.265387
129741,916,2023-01-01,amt_100_120,1,120.0,119.880120
129742,916,2023-01-01,amt_120_150,30,3775.0,125.829139


In [6]:
amt_grp=df_avg_price.pivot_table(values='avg_price', index = ['site_code','week_end_sun'],columns= 'GRIFOLS_TOTAL_PAID', aggfunc= 'sum').reset_index()
amt_grp.fillna(0, inplace=True)
amt_grp

GRIFOLS_TOTAL_PAID,site_code,week_end_sun,amt_0_30,amt_30_40,amt_40_50,amt_50_80,amt_80_100,amt_100_120,amt_120_150,amt_150+
0,5,2022-01-02,29.900891,36.663611,49.950050,79.986669,99.999487,110.188479,132.755769,179.778700
1,5,2022-01-09,29.939100,37.481259,0.000000,79.920080,99.999730,104.947526,132.516327,230.953809
2,5,2022-01-16,29.963431,37.137552,0.000000,79.996800,99.987313,105.412274,133.354810,252.249795
3,5,2022-01-23,29.999923,37.495313,49.950050,61.144559,99.999750,107.652886,131.674459,213.096100
4,5,2022-01-30,29.929672,36.662593,0.000000,79.994667,99.999701,108.569213,134.108247,186.930402
...,...,...,...,...,...,...,...,...,...,...
16213,916,2022-12-04,0.000000,39.999913,49.987503,74.962519,99.763358,116.627791,146.617794,177.801387
16214,916,2022-12-11,0.000000,39.161980,49.975012,74.962519,99.108666,113.295568,135.953310,180.438142
16215,916,2022-12-18,0.000000,39.317555,0.000000,0.000000,99.215683,0.000000,126.202545,181.981802
16216,916,2022-12-25,0.000000,39.196567,0.000000,62.437562,99.244971,0.000000,127.560832,166.638894


In [7]:
# step 2-- group by site_code and week_end_sun  
total_donations = df.groupby(['site_code','week_end_sun']).agg({'donations': ['sum']})
total_donations.columns = ['total_donations']
total_donations = total_donations.reset_index()
total_donations.head()

,site_code,week_end_sun,total_donations
0,5,2022-01-02,749
1,5,2022-01-09,936
2,5,2022-01-16,1029
3,5,2022-01-23,1107
4,5,2022-01-30,1168


In [8]:
# dependent variable-- total donation 
df_grp =amt_grp.merge(total_donations,on=['site_code','week_end_sun'],how='left')

In [9]:
# group by site_code and compute average and standard deviation of price
grouped = df_grp.groupby('site_code')
avg = grouped['total_donations'].transform('mean')
std = grouped['total_donations'].transform('std')

# compute dummy_pos column based on price, average, and standard deviation
df_grp['dummy_pos'] = (df_grp['total_donations'] >= (avg + 1.5 * std)).astype(int)
df_grp['dummy_neg'] = (df_grp['total_donations'] <= (avg - 1.5 * std)).astype(int)


In [10]:
# add 1 to each value and take the logarithm of the specified columns

cols_to_transform = ['amt_0_30', 'amt_30_40','amt_40_50','amt_50_80','amt_80_100','amt_100_120','amt_120_150','amt_150+','total_donations']
df_grp[cols_to_transform] = np.log(df_grp[cols_to_transform] + 1)



In [11]:
df_grp.head()

,site_code,week_end_sun,amt_0_30,amt_30_40,amt_40_50,amt_50_80,amt_80_100,amt_100_120,amt_120_150,amt_150+,total_donations,dummy_pos,dummy_neg
0,5,2022-01-02,3.430785,3.628694,3.930846,4.394285,4.615115,4.711227,4.896016,5.197274,6.620073,0,1
1,5,2022-01-09,3.432021,3.650171,0.000000,4.393462,4.615118,4.662944,4.894224,5.446538,6.842683,0,1
2,5,2022-01-16,3.432807,3.641199,0.000000,4.394410,4.614995,4.667321,4.900484,5.534376,6.937314,0,0
3,5,2022-01-23,3.433985,3.650536,3.930846,4.129463,4.615118,4.688158,4.887898,5.366425,7.010312,0,0
4,5,2022-01-30,3.431716,3.628667,0.000000,4.394383,4.615118,4.696556,4.906076,5.236072,7.063904,0,0


In [12]:
df_grp.isna().sum()
df_grp.fillna(0,inplace=True)

In [13]:
site_match=pd.read_csv("site_match.csv")
site_match['Week_ending_sunday']=pd.to_datetime(site_match['Week_ending_sunday'])

In [14]:

site_match.rename(columns = {'site code':'site_code'}, inplace = True)
site_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4091 entries, 0 to 4090
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Zipcode             4091 non-null   int64         
 1   Week_ending_sunday  4091 non-null   datetime64[ns]
 2   Precipitation(in)   4091 non-null   float64       
 3   site_code           4091 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 128.0 KB


In [15]:

df_grp['week_end_sun']=pd.to_datetime(df_grp['week_end_sun'])
final =df_grp.merge(site_match,left_on=['site_code','week_end_sun'],right_on=['site_code','Week_ending_sunday'],how='left')

In [16]:


#df_grp['site_code']=df_grp['site_code'].astype('object')
final['Precipitation(in)']=final['Precipitation(in)'].fillna(0)
final

,site_code,week_end_sun,amt_0_30,amt_30_40,amt_40_50,amt_50_80,amt_80_100,amt_100_120,amt_120_150,amt_150+,total_donations,dummy_pos,dummy_neg,Zipcode,Week_ending_sunday,Precipitation(in)
0,5,2022-01-02,3.430785,3.628694,3.930846,4.394285,4.615115,4.711227,4.896016,5.197274,6.620073,0,1,77031.0,2022-01-02,0.0
1,5,2022-01-09,3.432021,3.650171,0.000000,4.393462,4.615118,4.662944,4.894224,5.446538,6.842683,0,1,77031.0,2022-01-09,0.0
2,5,2022-01-16,3.432807,3.641199,0.000000,4.394410,4.614995,4.667321,4.900484,5.534376,6.937314,0,0,77031.0,2022-01-16,0.0
3,5,2022-01-23,3.433985,3.650536,3.930846,4.129463,4.615118,4.688158,4.887898,5.366425,7.010312,0,0,77031.0,2022-01-23,0.0
4,5,2022-01-30,3.431716,3.628667,0.000000,4.394383,4.615118,4.696556,4.906076,5.236072,7.063904,0,0,NaN,NaT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16213,916,2022-12-04,0.000000,3.713570,3.931581,4.330240,4.612775,4.767525,4.994626,5.186276,6.816736,1,0,21702.0,2022-12-04,0.0
16214,916,2022-12-11,0.000000,3.692921,3.931336,4.330240,4.606256,4.738788,4.919640,5.200915,6.799056,1,0,21702.0,2022-12-11,0.0
16215,916,2022-12-18,0.000000,3.696787,0.000000,0.000000,4.607325,0.000000,4.845781,5.209387,6.688355,0,0,21702.0,2022-12-18,0.0
16216,916,2022-12-25,0.000000,3.693782,0.000000,4.150056,4.607617,0.000000,4.856402,5.121812,6.587550,0,0,21702.0,2022-12-25,0.0


In [17]:
#### REGRESSION

site=final['site_code'].unique().tolist()

from sklearn.linear_model import LinearRegression   
list_dict=[]

for sites in site:
    df_test=final[final['site_code']==sites]
    result=df_test.loc[:,['amt_0_30','amt_30_40','amt_40_50','amt_50_80','amt_80_100', 'amt_100_120', 'amt_120_150', 'amt_150+','dummy_pos','dummy_neg','Precipitation(in)',
                        'total_donations']]
    X = result.drop('total_donations',axis=1) # Independet variable
    y = result['total_donations'] # dependent variable
    #X=X.add(1, fill_value = 1)
    #y=y.add(1, fill_value = 1)
    #X=np.log(X)
    #y=np.log(y)
    lm = LinearRegression(fit_intercept=False)
    lm.fit(X,y)
    m = lm.coef_
    for item in list(zip(X,m)):
        temp_dict = {}
        temp_dict["site"] = sites
        temp_dict["variables"] = item[0]
        temp_dict["coeff"] = item[1]
        
        
        list_dict.append(temp_dict)
       
        
Regression=pd.DataFrame(list_dict)     
   

In [18]:
Regression

,site,variables,coeff
0,5,amt_0_30,4.679685
1,5,amt_30_40,0.113686
2,5,amt_40_50,-0.004262
3,5,amt_50_80,0.027790
4,5,amt_80_100,0.749040
...,...,...,...
3361,916,amt_120_150,-0.023190
3362,916,amt_150+,0.013063
3363,916,dummy_pos,0.194231
3364,916,dummy_neg,-0.215473


In [20]:
##### UCM
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace import structural as smf

list_dict = []

for site_code in set(final['site_code']):
    df_test = final[final['site_code'] == site_code]
    
    dependent = df_test['total_donations']
    indep = df_test[['amt_0_30', 'amt_30_40', 'amt_40_50', 'amt_50_80', 'amt_80_100', 'amt_100_120', 'amt_120_150', 'amt_150+', 'dummy_pos', 'dummy_neg','Precipitation(in)']]
    
    ucm_init = smf.UnobservedComponents(endog=dependent,
                                        exog=indep,
                                        irregular=True,
                                        level=True,
                                        stochastic_level=True)
    
    ucm_init_fit = ucm_init.fit(cov_type='oim')
    
    vs_level = np.mean(ucm_init_fit.level.smoothed_cov)
    
    if vs_level < 1e-8:
        vs_level = np.std(dependent) / (len(df_test) - 1)
    
    ucm_init_1 = ucm_init.fit_constrained(constraints={'sigma2.level': vs_level * 0.1},
                                          cov_type='oim')
    
    estimates = ucm_init_1.params
    
    for item in zip(indep.columns, estimates):
        
        temp_dict = {}
        temp_dict["site"] = site_code
        temp_dict["variables"] = item[0]
        temp_dict["coeff"] = item[1]
        
        list_dict.append(temp_dict)
        
        
UCM=pd.DataFrame(list_dict)


C:\Users\GauravNegi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\GauravNegi\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\GauravNegi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\GauravNegi\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\GauravNegi\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood op

In [22]:
UCM.head(25)

,site,variables,coeff
0,512,amt_0_30,7.967644e-03
1,512,amt_30_40,2.289787e-06
2,512,amt_40_50,-3.286661e-02
3,512,amt_50_80,-9.901129e-03
4,512,amt_80_100,4.692375e-03
5,512,amt_100_120,2.778876e-02
6,512,amt_120_150,-1.960980e-01
7,512,amt_150+,-2.436375e-02
8,512,dummy_pos,-2.404243e-03
9,512,dummy_neg,-1.067639e-02


In [23]:
UCM.to_csv('UCM_result_v1.csv')
Regression.to_csv('Regression_result_v1.csv')